In [29]:
import json
import pandas as pd
import numpy as np

import re

from sqlalchemy import create_engine
import psycopg2

# from config import db_password

import time

import os

In [30]:
# 1. Create a function that takes in three arguments;
# Wikipedia data, Kaggle metadata, and MovieLens rating data (from Kaggle)

def extract_transform_load():
    # 2. Read in the kaggle metadata and MovieLens ratings CSV files as Pandas DataFrames.
    kaggle_metadata= pd.read_csv(kaggle_file)
    ratings= pd.read_csv(ratings_file)

    # 3. Open the read the Wikipedia data JSON file.
    with open(wiki_file, mode='r') as file:
        wiki_movies_raw = json.load(file)
    
    # 4. Read in the raw wiki movie data as a Pandas DataFrame.
    wiki_movies_df= pd.DataFrame(wiki_movies_raw)
    
    # 5. Return the three DataFrames
    return wiki_movies_df, kaggle_metadata, ratings

# 6 Create the path to your file directory and variables for the three files. 
file_dir = 'C:\\Users\Billy\Desktop\Bootcamp\Mods\Mod 8\Movies-ETL\Resources'

# Wikipedia data
wiki_file = os.path.join('Resources', 'wikipedia-movies.json')
# Kaggle metadata
kaggle_file = os.path.join('Resources', 'movies_metadata.csv')
# MovieLens rating data.
ratings_file = os.path.join('Resources', 'ratings.csv')

# 7. Set the three variables in Step 6 equal to the function created in Step 1.
wiki_file, kaggle_file, ratings_file = extract_transform_load()

C:\Users\Billy\AppData\Local\Temp/ipykernel_14340/359838319.py:30: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  wiki_file, kaggle_file, ratings_file = extract_transform_load()


In [31]:
# 8. Set the DataFrames from the return statement equal to the file names in Step 6. 
wiki_movies_df = wiki_file
kaggle_metadata = kaggle_file
ratings = ratings_file

In [ ]:
# 9. Check the wiki_movies_df DataFrame.
wiki_movies_df

In [ ]:
# 10. Check the kaggle_metadata DataFrame.
kaggle_metadata.head()

In [ ]:
# 11. Check the ratings DataFrame.
ratings.head()